# Initialization

## Installations

In [1]:
!pip install pandasql
!pip install pandas
!pip install plotly
!pip install optimparallel
!pip install torch
!pip install tensorflow
!pip install tensorflow_probability


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


## Imports

In [272]:
import pandas as pd
import numpy as np
import functools
import pickle

from BB_solver import bb_ll

from bs4 import BeautifulSoup
from aia import AIASession
from urllib.request import urlopen
from scipy.optimize import Bounds
from optimparallel import minimize_parallel

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier, StackingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import xgboost as XGBoost

## Reading the data

In [457]:
ncaa_2011_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_11.csv')
ncaa_2012_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_12.csv')
ncaa_2013_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_13.csv')
ncaa_2014_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_14.csv')
ncaa_2015_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_15.csv')
ncaa_2016_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_16.csv')
ncaa_2017_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_17.csv')
ncaa_2018_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_18.csv')
ncaa_2019_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_19.csv')
ncaa_2021_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_21.csv')
ncaa_2022_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_22.csv')

# Data Wrangling

## Feature Selection

We are training on % metrics (e.g., how predictive of winning was a high 3-point shooting percentage)
to determine which features contribute most towards winning a single game.
We are predicting on the beta binomial alpha/beta parameter values for both teams.
We will standard scale test data on the "_avgs_" metrics with consist of $\frac{alpha}{alpha+beta}$.
Obviously, we don't know the in-game stats for the contest we are trying to predict.

In [458]:
filter = [
    # 'Team',
    # 'Rating', 'OppRating',
    'Win',
    'EFG_alphas_team1', 'EFG_betas_team1', 'EFG_avgs_team1',
    'EFG2_alphas_team1', 'EFG2_betas_team1', 'EFG2_avgs_team1',
    'EFG3_alphas_team1', 'EFG3_betas_team1', 'EFG3_avgs_team1',
    # 'EFT_alphas_team1', 'EFT_betas_team1', 'EFT_avgs_team1',
    'EORB_alphas_team1', 'EORB_betas_team1', 'EORB_avgs_team1',
    # 'EDRB_alphas_team1', 'EDRB_betas_team1', 'EDRB_avgs_team1',
    'EAST_alphas_team1', 'EAST_betas_team1', 'EAST_avgs_team1',
    'ETOV_alphas_team1', 'ETOV_betas_team1', 'ETOV_avgs_team1',
    'EFG_alphas_team2', 'EFG_betas_team2', 'EFG_avgs_team2',
    'EFG2_alphas_team2', 'EFG2_betas_team2', 'EFG2_avgs_team2',
    'EFG3_alphas_team2', 'EFG3_betas_team2', 'EFG3_avgs_team2',
    'EORB_alphas_team2', 'EORB_betas_team2', 'EORB_avgs_team2',
    # 'EDRB_alphas_team2', 'EDRB_betas_team2', 'EDRB_avgs_team2',
    'EAST_alphas_team2', 'EAST_betas_team2', 'EAST_avgs_team2',
    'ETOV_alphas_team2', 'ETOV_betas_team2', 'ETOV_avgs_team2',
    ]

filter_predict = [
    # 'Team',
    # 'Rating', 'OppRating',
    'EFG_avgs_team1',
    'EFG2_avgs_team1',
    'EFG3_avgs_team1',
    # 'EFT_avgs_team1',
    'EORB_avgs_team1',
    # 'EDRB_avgs_team1',
    'EAST_avgs_team1',
    'ETOV_avgs_team1',
    'EBLK_avgs_team1',

    'EFG_avgs_team2',
    'EFG2_avgs_team2',
    'EFG3_avgs_team2',
    # 'EFT_avgs_team2',
    'EORB_avgs_team2',
    # 'EDRB_avgs_team2',
    'EAST_avgs_team2',
    'ETOV_avgs_team2',
    'EBLK_avgs_team2'
]

filter_train = [
    # 'Team',
    # 'Rating', 'OppRating',
    'EFG%_team1',
    'EFG2%_team1',
    'EFG3%_team1',
    # 'EFT%_team1',
    'EORB%_team1',
    # 'EDRB%_team1',
    'EAST%_team1',
    'ETOV%_team1',
    'EBLK%_team1',

    'EFG%_team2',
    'EFG2%_team2',
    'EFG3%_team2',
    # 'EFT%_team2',
    'EORB%_team2',
    # 'EDRB%_team2',
    'EAST%_team2',
    'ETOV%_team2',
    'EBLK%_team2',
]

## Train Test Split (MM Tournament vs. Regular Season)

In [459]:
ncaa_tour_2011_df = ncaa_2011_df[ncaa_2011_df['DayNum'] > 135].drop_duplicates(subset=['GameId'])
ncaa_tour_2012_df = ncaa_2012_df[ncaa_2012_df['DayNum'] > 135].drop_duplicates(subset=['GameId'])
ncaa_tour_2013_df = ncaa_2013_df[ncaa_2013_df['DayNum'] > 135].drop_duplicates(subset=['GameId'])
ncaa_tour_2014_df = ncaa_2014_df[ncaa_2014_df['DayNum'] > 135].drop_duplicates(subset=['GameId'])
ncaa_tour_2015_df = ncaa_2015_df[ncaa_2015_df['DayNum'] > 135].drop_duplicates(subset=['GameId'])
ncaa_tour_2016_df = ncaa_2016_df[ncaa_2016_df['DayNum'] > 135].drop_duplicates(subset=['GameId'])
ncaa_tour_2017_df = ncaa_2017_df[ncaa_2017_df['DayNum'] > 135].drop_duplicates(subset=['GameId'])
ncaa_tour_2018_df = ncaa_2018_df[ncaa_2018_df['DayNum'] > 135].drop_duplicates(subset=['GameId'])
ncaa_tour_2019_df = ncaa_2019_df[ncaa_2019_df['DayNum'] > 135].drop_duplicates(subset=['GameId'])
ncaa_tour_2021_df = ncaa_2021_df[ncaa_2021_df['DayNum'] > 136].drop_duplicates(subset=['GameId'])
ncaa_tour_2022_df = ncaa_2022_df[ncaa_2022_df['DayNum'] > 135].drop_duplicates(subset=['GameId'])

In [460]:
ncaa_2011_df = ncaa_2011_df[ncaa_2011_df['DayNum'] <= 135]
ncaa_2012_df = ncaa_2012_df[ncaa_2012_df['DayNum'] <= 135]
ncaa_2013_df = ncaa_2013_df[ncaa_2013_df['DayNum'] <= 135]
ncaa_2014_df = ncaa_2014_df[ncaa_2014_df['DayNum'] <= 135]
ncaa_2015_df = ncaa_2015_df[ncaa_2015_df['DayNum'] <= 135]
ncaa_2016_df = ncaa_2016_df[ncaa_2016_df['DayNum'] <= 135]
ncaa_2017_df = ncaa_2017_df[ncaa_2017_df['DayNum'] <= 135]
ncaa_2018_df = ncaa_2018_df[ncaa_2018_df['DayNum'] <= 135]
ncaa_2019_df = ncaa_2019_df[ncaa_2019_df['DayNum'] <= 135]
ncaa_2021_df = ncaa_2021_df[ncaa_2021_df['DayNum'] <= 136]
ncaa_2022_df = ncaa_2022_df[ncaa_2022_df['DayNum'] <= 135]

## Feature Engineering

In [461]:
# concatenate the regular season game data
frames = [ncaa_2011_df, ncaa_2012_df, ncaa_2013_df, ncaa_2014_df,
          ncaa_2015_df, ncaa_2016_df, ncaa_2017_df, ncaa_2018_df,
          ncaa_2019_df, ncaa_2021_df, ncaa_2022_df]
combined_df = pd.concat(frames)

# get training gold labels – regular season winners
y_train = combined_df['Win']

combined_df['EFG%_team1'] = combined_df['EFG_N'] / combined_df['EFG_D']
combined_df['EFG2%_team1'] = combined_df['EFG2_N'] / combined_df['EFG2_D']
combined_df['EFG3%_team1'] = combined_df['EFG3_N'] / combined_df['EFG3_D']
# combined_df['EFT%_team1'] = combined_df['EFT_N'] / combined_df['EFT_D']
combined_df['EORB%_team1'] = combined_df['EORB_N'] / combined_df['EORB_D']
# combined_df['EDRB%_team1'] = combined_df['EDRB_N'] / combined_df['EDRB_D']
combined_df['EAST%_team1'] = combined_df['EAST_N'] / combined_df['EAST_D']
combined_df['ETOV%_team1'] = combined_df['ETOV_N'] / combined_df['ETOV_D']
combined_df['EBLK%_team1'] = combined_df['EBLK_N'] / combined_df['EBLK_D']

combined_df['EFG%_team2'] = combined_df['EFG_N_Opp'] / combined_df['EFG_D_Opp']
combined_df['EFG2%_team2'] = combined_df['EFG2_N_Opp'] / combined_df['EFG2_D_Opp']
combined_df['EFG3%_team2'] = combined_df['EFG3_N_Opp'] / combined_df['EFG3_D_Opp']
# combined_df['EFT%_team2'] = combined_df['EFT_N_Opp'] / combined_df['EFT_D_Opp']
combined_df['EORB%_team2'] = combined_df['EORB_N_Opp'] / combined_df['EORB_D_Opp']
# combined_df['EDRB%_team2'] = combined_df['EDRB_N_Opp'] / combined_df['EDRB_D_Opp']
combined_df['EAST%_team2'] = combined_df['EAST_N_Opp'] / combined_df['EAST_D_Opp']
combined_df['ETOV%_team2'] = combined_df['ETOV_N_Opp'] / combined_df['ETOV_D_Opp']
combined_df['EBLK%_team2'] = combined_df['EBLK_N_Opp'] / combined_df['EBLK_D_Opp']

# instantiate standard scalers
predict_scaler = StandardScaler()
# get scaled test data
predict_scaler.fit(combined_df[filter_predict])

train_scaler = StandardScaler()
# get scaled training data
scaled_X_train = train_scaler.fit_transform(combined_df[filter_train])

# concatenate the MM tournament game data
frames = [ncaa_tour_2011_df, ncaa_tour_2012_df, ncaa_tour_2013_df, ncaa_tour_2014_df,
          ncaa_tour_2015_df, ncaa_tour_2016_df, ncaa_tour_2017_df, ncaa_tour_2018_df,
          ncaa_tour_2019_df, ncaa_tour_2021_df, ncaa_tour_2022_df]
combined_tour_df = pd.concat(frames)

# get test gold labels – MM tournament season winners
y_test = combined_tour_df['Win']

# scale the data
scaled_X_test = predict_scaler.transform(combined_tour_df[filter_predict])
scaled_X_test_11 = predict_scaler.transform(ncaa_tour_2011_df[filter_predict])
scaled_X_test_12 = predict_scaler.transform(ncaa_tour_2012_df[filter_predict])
scaled_X_test_13 = predict_scaler.transform(ncaa_tour_2013_df[filter_predict])
scaled_X_test_14 = predict_scaler.transform(ncaa_tour_2014_df[filter_predict])
scaled_X_test_15 = predict_scaler.transform(ncaa_tour_2015_df[filter_predict])
scaled_X_test_16 = predict_scaler.transform(ncaa_tour_2016_df[filter_predict])
scaled_X_test_17 = predict_scaler.transform(ncaa_tour_2017_df[filter_predict])
scaled_X_test_18 = predict_scaler.transform(ncaa_tour_2018_df[filter_predict])
scaled_X_test_19 = predict_scaler.transform(ncaa_tour_2019_df[filter_predict])
scaled_X_test_21 = predict_scaler.transform(ncaa_tour_2021_df[filter_predict])
scaled_X_test_22 = predict_scaler.transform(ncaa_tour_2022_df[filter_predict])

# get the
y_test_11 = ncaa_tour_2011_df['Win']
y_test_12 = ncaa_tour_2012_df['Win']
y_test_13 = ncaa_tour_2013_df['Win']
y_test_14 = ncaa_tour_2014_df['Win']
y_test_15 = ncaa_tour_2015_df['Win']
y_test_16 = ncaa_tour_2016_df['Win']
y_test_17 = ncaa_tour_2017_df['Win']
y_test_18 = ncaa_tour_2018_df['Win']
y_test_19 = ncaa_tour_2019_df['Win']
y_test_21 = ncaa_tour_2021_df['Win']
y_test_22 = ncaa_tour_2022_df['Win']

Check dataframe.

In [462]:
combined_df[combined_df['Team'] == 'Duke'][['DayNum', 'EFG_avgs_team1', 'EFG%_team1', 'EFG_avgs_team1',
                                            'EFG2_avgs_team1',
                                            'EFG3_avgs_team1',]]

,DayNum,EFG_avgs_team1,EFG%_team1,EFG_avgs_team1,EFG2_avgs_team1,EFG3_avgs_team1
318,13,0.537441,0.661538,0.537441,0.537719,0.362724
488,15,0.568179,0.500000,0.568179,0.557546,0.413000
740,18,0.594820,0.555556,0.594820,0.598715,0.391232
1097,21,0.543681,0.593750,0.543681,0.529125,0.356199
1185,22,0.504547,0.537037,0.504547,0.498119,0.347783
...,...,...,...,...,...,...
9709,120,0.573556,0.703704,0.573556,0.552437,0.389918
10090,124,0.554556,0.550725,0.554556,0.544149,0.379572
10459,129,0.547521,0.567164,0.547521,0.569000,0.342069
10573,130,0.583303,0.551724,0.583303,0.597503,0.352018


## Hyperparameter-Tuning

We'll use grid search with k=3 folds cross validation

In [464]:
def ParameterSearch(regressor, degree, scaler_bool, interaction_only, X_train, y_train):
    """
    This is a k=5 cross fold validation function to find optimal parameters for
    different regressors.

    Parameters:
    ------------
    regressor: (string)
      type or regressor to be run ("rfr", "etr", "gbr", "xgb", "knnr", "ablr",
                                   "svr", "lr")
    scaler_bool: bool
      True to apply standard scaler to the input data first
    degree: int
      degree to which the X_train input will be converted prior to running the
      regressor
    interaction_only: bool
      if the X_train input degree will only add interaction terms
    X_train: DataFrame
      training inputs
    y_train: DataFrame
      training outputs

    Returns:
    ------------
    None
    """

    # all of the regressor options to be run
    rfc = RandomForestClassifier(random_state=42, n_jobs = -1)
    etc = ExtraTreesClassifier(random_state=42, n_jobs = -1)
    nnc = MLPClassifier(random_state=42,  max_iter=10000)
    gbc = GradientBoostingClassifier(random_state=42)
    xgb = XGBoost.XGBClassifier(random_state=42, n_jobs=-1)
    knnc = KNeighborsClassifier()
    absvc = AdaBoostClassifier(random_state=42, base_estimator=SVC())
    svc = SVC()
    lr = LogisticRegression(random_state=0, warm_start=True, max_iter=10000)

    # parameter values to be tried
    rfc_grid = {
        'rfc__max_depth': [4, 6, 8, 10],
        'rfc__max_features': ['sqrt', None],
        'rfc__min_samples_leaf': [4, 8, 16, 32],
        'rfc__n_estimators': [50, 100, 200],
        'rfc__bootstrap': [True],
        'rfc__max_samples': [0.25, 0.5, 1]
    }
    etc_grid = {
        'etc__max_depth': [4, 6, 8, 10],
        'etc__max_features': ['sqrt', None],
        'etc__min_samples_leaf': [4, 8, 16, 32],
        'etc__n_estimators': [50, 100, 200],
        'etc__bootstrap': [True],
        'etc__max_samples': [0.25, 0.5, 1]
    }
    nnc_grid = {
        'nnc__hidden_layer_sizes': [(i, j) for i in range(5, 51, 5) for j in range(2, 21, 2)],
        'nnc__alpha': [i/2 for i in list(range(20))],
    }
    gbc_grid = {
        'gbc__max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
        'gbc__max_features': ['sqrt', None],
        'gbc__min_samples_leaf': [1, 2, 4, 8, 16],
        'gbc__n_estimators': [50, 100, 200, 300],
        'gbc__subsample': [0.25, 0.5, 0.75, 1]
    }
    xgb_grid = {
        'xgb__max_depth': [5, 10, 20],
        'xgb__n_estimators': [100, 200, 300],
        'xgb__gamma': [1],
        'xgb__eta': [0.05, 0.1],
        'xgb__subsample': [0.25],
        'xgb__min_child_weight': [0.25, 0.5],
        'xgb__reg_lambda': [0.5, 1],
        'xgb__alpha': [0.5, 1]
    }
    knnc_grid = {
        'knnc__n_neighbors': list(range(1, 51, 1))
    }
    absvc_grid = {
        'absvc__n_estimators': list(range(10, 101, 10)),
        'absvc__learning_rate': [0.05, 0.1, 0.2]
    }
    svc_grid = {
        'svc__degree': list(range(1, 11, 1))
    }
    lr_grid = {
        'lr__C': [0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25, 3.5, 3.75, 4, 4.25, 4.5, 4.75, 5]
    }

    # check if scaler will be used
    if scaler_bool == True:
        scaler = StandardScaler()
    else:
        scaler = None

    if degree:
        poly = PolynomialFeatures(degree=degree, interaction_only=interaction_only)
    else:
        poly = None

    # set up the grid and pipe for the chosen regressor
    if regressor=='rfc':
        grid = rfc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('rfc', rfc)])
    elif regressor=='etc':
        grid = etc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('etc', etc)])
    elif regressor=='nnc':
        grid = nnc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('nnc', nnc)])
    elif regressor=='gbc':
        grid = gbc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('gbc', gbc)])
    elif regressor=='xgb':
        grid = xgb_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('xgb', xgb)])
    elif regressor=='knnc':
        grid = knnc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('knnc', knnc)])
    elif regressor=='absvc':
        grid = absvc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('absvc', absvc)])
    elif regressor=='svc':
        grid = svc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('svc', svc)])
    elif regressor=='lr':
        grid = lr_grid
        pipe = Pipeline(steps=[('poly', poly), ('scaler', scaler), ('lr', lr)])

    # create the GridSearchCV object, and fit the passed input to it
    gs = GridSearchCV(pipe, grid, n_jobs=-1, cv=3, verbose=1)
    gs.fit(X_train, y_train)

    # print out the best score on the cross validation holdout data and the best parameters
    print("Best score:", gs.best_score_)
    print()
    print(gs.best_params_)

In [467]:
ParameterSearch('lr', 2, True, False, scaled_X_train, y_train)

Fitting 3 folds for each of 26 candidates, totalling 78 fits


/Users/kylesullivan/anaconda3/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best score: 0.9186218668896204

{'lr__C': 2.5}


In [485]:
ParameterSearch('rfc', 1, False, False, scaled_X_train, y_train)

Fitting 3 folds for each of 288 candidates, totalling 864 fits
Best score: 0.8995227395446865

{'rfc__bootstrap': True, 'rfc__max_depth': 10, 'rfc__max_features': None, 'rfc__max_samples': 0.25, 'rfc__min_samples_leaf': 4, 'rfc__n_estimators': 200}


In [36]:
ParameterSearch('nnc', 1, False, False, scaled_X_train, y_train)

Fitting 3 folds for each of 2000 candidates, totalling 6000 fits
Best score: 0.9179686819599032

{'nnc__alpha': 0.5, 'nnc__hidden_layer_sizes': (45, 8)}


In [466]:
ParameterSearch('xgb', 1, False, False, scaled_X_train, y_train)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


/Users/kylesullivan/anaconda3/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best score: 0.915477870094581

{'xgb__alpha': 0.5, 'xgb__eta': 0.1, 'xgb__gamma': 1, 'xgb__max_depth': 5, 'xgb__min_child_weight': 0.5, 'xgb__n_estimators': 300, 'xgb__reg_lambda': 1, 'xgb__subsample': 0.25}


In [490]:
def EnsemblePredictor(X_train, y_train, X_test=None, y_test=None):
    """
    An ensemble regressor estimation function.

    Parameters:
    ------------
    X_train: DataFrame
      training inputs
    y_train: DataFrame
      training outputs
    X_test: DataFrame
      testing inputs
    y_test: DataFrame
      testing outputs

    Returns:
    ------------
    None (if y_test != None)

    """

    # normalize the data
    scaler = StandardScaler()

    poly_class=Pipeline([('poly_class', PolynomialFeatures(degree=2, interaction_only=False)), ('scaler', scaler), ('lr', LogisticRegression(random_state=10, warm_start=True, C=2.5, max_iter=20000),)])

    # RF_class=Pipeline([('scaler', scaler), ('RF_class', RandomForestClassifier(bootstrap=True, max_depth=10, max_features=None, max_samples=0.25, min_samples_leaf=4, n_estimators=200, random_state=0),)])

    XGB_class=Pipeline([('scaler', scaler), ('XGB_class', XGBoost.XGBClassifier(alpha=0.5, eta=0.1, gamma=1, max_depth=5, min_child_weight=0.5, n_estimators=300, reg_lambda=1, subsample=0.25, random_state=42, n_jobs=-1),)])

    # estimators to be used in the ensemble
    estimators = [
        # ('0', XGB_class,),
        # ('1', RF_class,),
        ('2', poly_class,),
        ]

    # create regressors stack
    stack_reg = StackingClassifier(
        estimators=estimators
    )

    # fit the training data to the stack
    stack_reg.fit(X_train, y_train)

    train_acc = stack_reg.score(X_train, y_train)
    train_pred = stack_reg.predict(X_train)
    y_train_arr = np.array(y_train)

    print("Train Acc:", round(train_acc, 2))
    if not y_test is None:
        test_acc = stack_reg.score(X_test, y_test)
        print("Test Acc:", round(test_acc, 2))
    print('Train Total Incorrect:', len(train_pred)-sum(train_pred==y_train_arr))
    if not y_test is None:
        y_test_arr = np.array(y_test)
        print('Test Total Correct:', sum(test_pred==y_test_arr))
    elif not X_test is None:
        return test_pred_prob[:,1]
    else:
        print("--------------------------------------------------",)
        total=0
        for pair in [(scaled_X_test_11, y_test_11, 2011), (scaled_X_test_12, y_test_12, 2012), (scaled_X_test_13, y_test_13, 2013),
                     (scaled_X_test_14, y_test_14, 2014), (scaled_X_test_15, y_test_15, 2015), (scaled_X_test_16, y_test_16, 2016),
                     (scaled_X_test_17, y_test_17, 2017), (scaled_X_test_18, y_test_18, 2018), (scaled_X_test_19, y_test_19, 2019),
                     (scaled_X_test_21, y_test_21, 2021), (scaled_X_test_22, y_test_22, 2022)]:
            test_pred = stack_reg.predict(pair[0])
            y_train_arr = np.array(pair[1])
            current = sum(test_pred==y_train_arr)
            total+=current
            print(pair[2], 'Test Correct:', current)
            print("--------------------------------------------------",)
        print('Total Correct:', total)

Logistic Regression Results (ensemble of one):

In [491]:
EnsemblePredictor(scaled_X_train, y_train)

Train Acc: 0.92
Train Total Incorrect: 9306
--------------------------------------------------
2011 Test Correct: 42
--------------------------------------------------
2012 Test Correct: 46
--------------------------------------------------
2013 Test Correct: 45
--------------------------------------------------
2014 Test Correct: 43
--------------------------------------------------
2015 Test Correct: 50
--------------------------------------------------
2016 Test Correct: 46
--------------------------------------------------
2017 Test Correct: 45
--------------------------------------------------
2018 Test Correct: 44
--------------------------------------------------
2019 Test Correct: 48
--------------------------------------------------
2021 Test Correct: 41
--------------------------------------------------
2022 Test Correct: 42
--------------------------------------------------
Total Correct: 492
